# Career Conversation Project

In [41]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [42]:
load_dotenv(override=True)
openai = OpenAI()
gemini = OpenAI(
    api_key = os.getenv('GOOGLE_API_KEY'),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [43]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [44]:
reader = PdfReader("../me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("../me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Adriana Salcedo"

In [45]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

## Tools

In [46]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}


def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

def record_personal_question(question, acceptable):
    if acceptable:
        push(f'A personal question was asked and answered:\n {question}')
    else:
        push(f'A personal question was asked and not answered:\n {question}')
    return {"recorded": "ok"}
    

def record_skill_question(question):
    push(f'A skill-related question was asked:\n {question}')
    return {'recorded': 'ok'}

In [47]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [48]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [49]:
record_personal_question_json = {
    'name': 'record_personal_question',
    'description': 'Use this tool to log if a personal question was asked. Indicate if the question is acceptable (can be answered) or not.',
    'parameters': {
        'type': 'object',
        'properties': {
            'question': {
                'type': 'string',
                'description': 'Question that will not be answered'
            },
            'acceptable': {
                'type': 'boolean',
                'description': 'Indicates if a question is acceptable or not'
            }
        },
        'required': ['question', 'acceptable'],
        'additionalProperties': False
    }
}

In [50]:
record_skill_question_json = {
    'name': 'record_skill_question',
    'description': (
        "Whenever a user asks about any skill, technology, tool, programming language, or experience"
        "regardless of whether it is present in the profile or not. ALWAYS use this tool to notify the owner. "
        "Pass the original user question as the argument. "
        "Examples: 'Do you know Python?', 'Have you worked with Tableau?', 'Are you familiar with cloud computing?'"
    ),
    'parameters': {
        'type': 'object',
        'properties': {
            'question': {
                'type': 'string',
                'description': 'Skill-related question was asked'
            },
        },
        'required': ['question'],
        'additionalProperties': False
    }
}


In [51]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json},
        {'type': 'function', 'function': record_personal_question_json},
        {'type': 'function', 'function': record_skill_question_json}
        
]

In [52]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [53]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

## Implement Evaluator

In [54]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [55]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback if necessary."

In [56]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable."
    return user_prompt

In [57]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [58]:
def push_evaluation(question, answer, evaluation):
    message_text = (
        f'New Evaluation:\n'
        f'Question: {question}\n'
        f'Agent answer: {answer}\n'
        f'Evaluation: {'is acceptable' if evaluation.is_acceptable else 'not acceptable'}'
    )

    #print("MESSAGE TEXT FÜR PUSH:", message_text)
    
    payload = {"user": pushover_user, 
               "token": pushover_token, 
               "message": message_text}
    requests.post(pushover_url, data=payload)

In [59]:
def chat(message, history):

    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason


        # Tool Calls
        if finish_reason=="tool_calls":
            msg = response.choices[0].message
            tool_calls = msg.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(msg)
            messages.extend(results)

            response_final = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
            agent_reply = response_final.choices[0].message.content

            # Evaluation
            evaluation = evaluate(agent_reply, message, history)
            push_evaluation(message, agent_reply, evaluation)

            return agent_reply
            
        else:
            done = True
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()